In [93]:
import pandas as pd
import numpy as np
import random

def clean_separate_df(df, yr):
    all_players = df[df.year==yr][['player','pos','adp','ppr_pts','year']].sort_values('adp').reset_index(drop=True)
    qb = all_players[all_players.pos=='QB']
    rb = all_players[all_players.pos=='RB']
    wr = all_players[all_players.pos=='WR']
    te = all_players[all_players.pos=='TE']
    return all_players, qb, rb, wr, te


def create_strats(strat):
    if strat == 'h':
        return ['hero_rb', ['RB','TE','QB','WR','WR','WR','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'z':
        return ['zero_rb', ['WR','WR','TE','QB','VAL','VAL','RB','RB','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'r':
        return ['rb_heavy', ['RB','RB','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    else:
        return ['val',['VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    
def create_owners_strats(hero,zero,rb_heavy,value):
    strats = []
    for i in range(0,hero):
        strats.append(create_strats('hero'))
    for i in range(0,zero):
        strats.append(create_strats('zero'))
    for i in range(0,rb_heavy):
        strats.append(create_strats('rb_heavy'))
    for i in range(0,value):
        strats.append(create_strats('value'))
    return strats

def one_random_strat():
    '''h - hero, z - zero, r - rb_heavy, v - value'''
    return random.choice(['h','z','r','v','v','v'])

def create_league():
    league_draft_strat= []
    for i in range(0,12):
        league_draft_strat.append(one_random_strat())
        
    strats = []
    for l in league_draft_strat:
        strats.append(create_strats(l))
        
    return strats

class Cheatsheet:
    def __init__(self, all_players, qb, rb, wr, te):
        self.all_players = all_players
        self.qb = qb
        self.rb = rb
        self.wr = wr
        self.te = te
      
    def draft_qb(self,owner):
        selection = self.qb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.qb = self.qb[1:]
        owner.qb_count+=1
        return selection
        
    def draft_rb(self,owner):
        selection = self.rb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.rb = self.rb[1:]
        owner.rb_count+=1
        return selection
    
    def draft_wr(self,owner):
        selection = self.wr.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.wr = self.wr[1:]
        owner.wr_count+=1
        return selection
    
    def draft_te(self,owner):
        selection = self.te.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.te = self.te[1:]
        owner.te_count+=1
        return selection
    
    def draft_not_qb(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'QB':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_te(owner)
    
    def draft_not_te(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'TE':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_qb(owner)
        
    def draft_not_wr(self,owner):
        pass
    
    def draft_value(self,owner): 
        selection = self.all_players.iloc[0,:]
        if self.all_players.iloc[0,:].pos == 'QB':
            self.qb = self.qb.drop(index = selection.name)
            owner.qb_count+=1
        elif self.all_players.iloc[0,:].pos == 'RB':
            self.rb = self.rb.drop(index = selection.name)
            owner.rb_count+=1
        elif self.all_players.iloc[0,:].pos == 'WR':
            self.wr = self.wr.drop(index = selection.name)
            owner.wr_count+=1
        else:
            self.te = self.te.drop(index = selection.name)
            owner.te_count+=1
        self.all_players = self.all_players.iloc[1:]
        return selection
    

class Team_owner:
    def __init__(self, strat, draft_pos, draft_num):
        self.team = []
        self.strat_name = strat[0]      
        self.strat_order = strat[1]
        self.draft_pos = draft_pos
        self.draft_num = draft_num
        self.qb_count = 0
        self.te_count = 0
        self.wr_count = 0
        self.rb_count = 0
        
        
    def draft_player(self,cheatsheet):
        if len(self.team) == 13 and self.te_count == 0: #Draft a TE in 10th round if no TEs
            self.strat_order.pop(0)
            return cheatsheet.draft_te(self)
        
        elif len(self.team) == 8 and self.qb_count == 0: #Draft a QB in 9th round if no QBs
            self.strat_order.pop(0)
            return cheatsheet.draft_qb(self)
        
        elif len(self.team) == 7 and self.rb_count == 1: #Draft a RB in 8th round if only 1 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 1: #Draft a WR in 8th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif len(self.team) == 6 and self.rb_count == 0: #Draft a RB in 7th round if only 0 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 0: #Draft a WR in 7th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif self.qb_count == 2 and self.strat_order[0]=='VAL': #Don't draft more than 2 QB
            self.strat_order.pop(0)
            return cheatsheet.draft_not_qb(self)
        
        elif self.te_count == 2 and self.strat_order[0]=='VAL': # Don't draft more than 2 TE
            self.strat_order.pop(0)
            return cheatsheet.draft_not_te(self)
        
        
#         elif self.rb_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_wr(self)
        
#         elif self.wr_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_rb(self)
        
        else:
            pos = self.strat_order.pop(0)
            if pos == 'QB':
                self.qb_count+=1
                return cheatsheet.draft_qb(self)
            elif pos == 'RB':
                return cheatsheet.draft_rb(self)
            elif pos == 'WR':
                return cheatsheet.draft_wr(self)
            elif pos == 'TE':
                return cheatsheet.draft_te(self)
            else:
                return cheatsheet.draft_value(self)


def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    top7 = [] 
    for season in range(2010,2023):
        for sim in range(0,100):
            for i in range(1,13):
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='QB')].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & ((df.pos=='RB')|(df.pos=='WR'))].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='TE')].sort_values('ppr_pts',ascending=False).head(1))
    return pd.concat(top7)
            
    

# Create 100 mock leagues each year to run analysis on

In [81]:
df = pd.read_csv('season1.csv',index_col=0)
owners_teams = [] 
for season in range(2010,2023):
    for sim in range(0,100):
        all_players, qb, rb, wr, te = clean_separate_df(df,season)
        strats = create_league()
        cheat = Cheatsheet(all_players, qb, rb, wr, te)

        owners = [] #Create 12 team owners with 'value_strat' drafting strategy
        for i, s in zip(range(0,12),strats.copy()):
            owners.append(Team_owner(s,i+1,sim))

        owners.reverse()
        for i in range(0,14): #Draft 14 rounds for each team owner
            owners.reverse()
            for o in owners:
                o.team.append(o.draft_player(cheat))

        owners_teams.append(owners)

all_teams = []
for t in owners_teams:
    for o in t:
        df = pd.concat(o.team,axis=1).T
        df['draft_pos'] = o.draft_pos
        df['draft_strat'] = o.strat_name
        df['draft_num'] = o.draft_num
        all_teams.append(df) 

df = pd.concat(all_teams)
    
   

In [127]:
top = top7(df)

In [132]:
top[(top.year==2022) & (top.draft_pos==12)& (top.draft_num==99)]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num


In [125]:
df[(df.year==2022) & (df.draft_pos==12)& (df.draft_num==99)]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
0,Jonathan Taylor,RB,1.0,146.4,2022,12,hero_rb,99
39,George Kittle,TE,40.0,200.5,2022,12,hero_rb,99
40,Jalen Hurts,QB,41.0,378.0,2022,12,hero_rb,99
45,DeAndre Hopkins,WR,48.0,151.7,2022,12,hero_rb,99
49,Jaylen Waddle,WR,52.0,259.2,2022,12,hero_rb,99
76,DeVonta Smith,WR,81.0,254.6,2022,12,hero_rb,99
72,Dalton Schultz,TE,76.0,142.7,2022,12,hero_rb,99
94,James Cook,RB,100.0,105.7,2022,12,hero_rb,99
95,Zach Ertz,TE,101.0,115.6,2022,12,hero_rb,99
119,Chris Olave,WR,129.0,198.2,2022,12,hero_rb,99


In [126]:
def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    top = []
    for season in range(2010,2023):
        for sim in range(0,100):
            for i in range(1,13):
                top7 = [] 
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='QB')].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & ((df.pos=='RB')|(df.pos=='WR'))].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='TE')].sort_values('ppr_pts',ascending=False).head(1))
                top.append(pd.concat(top7))
    return pd.concat(top)

In [128]:
top[(top.year==2022) & (top.draft_pos==5) & (top.draft_num==5)]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num


In [104]:
top.groupby(['year','draft_num','draft_pos'])['ppr_pts'].sum()

year  draft_num  draft_pos
2010  0          1            1425.4
                 2            1260.5
                 3            1517.0
                 4            1485.4
                 5            1485.7
                               ...  
2022  86         9              86.1
      87         5              86.1
      88         11             86.1
      97         5              86.1
      99         5              86.1
Name: ppr_pts, Length: 4378, dtype: object

In [95]:
top.groupby('draft_pos').

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
12,Drew Brees,QB,13.0,270.2,2010,1,val,0
34,LeSean McCoy,RB,35.0,297.2,2010,1,val,0
35,Jahvid Best,RB,36.0,196.2,2010,1,val,0
11,Calvin Johnson,WR,12.0,266.2,2010,1,val,0
83,Santonio Holmes,WR,86.0,160.3,2010,1,val,0
